In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def Prepr_1_Dish_Ingr(dir_source, dir_save):
    
    dishes = open(dir_source+'train_images.txt','r')
    index = open(dir_source+'train_labels.txt','r')
    ingredients = open(dir_source+'ingredients_simplified_Recipes5k.txt','r')

    dish_ingr_rel = open(dir_save+'dish_ingredient_relation.txt','w')

    dish_name = dishes.readlines()
    idxs = index.readlines()
    dish_ingredients = ingredients.readlines() 
    
    for i in range(len(idxs)):
        
        dish = dish_name[i].split('/')[0]
        #print(dish)
        ingredients = dish_ingredients[int(idxs[i])-1].replace('\n','').split(',')
        
        for ingr in ingredients:
            
            dish_ingr_rel.write(str(dish)+','+str(ingr)+'\n')
    
    
    dish_ingr_rel.close()
    
def Prepr_2_Dish_Ingr_Occ(directory,dir_save):
    
    dish_ingr_df = pd.read_csv(directory+'dish_ingredient_relation.txt',sep=',',names=['Dish','Ingredient'])
    dish_ingr_df['Occurrence'] = 1
    dish_ingr_df = dish_ingr_df.groupby(['Dish','Ingredient']).count()
    dish_ingr_df.to_csv(dir_save+'dish_ingredient_occurrences.csv')
    
def tuple_dish_ingr(directory, dir_save):

    dishes = open(directory+'train_images.txt','r')
    index = open(directory+'train_labels.txt','r')
    ingredients = open(directory+'ingredients_simplified_Recipes5k.txt','r')
    
    dish_name = dishes.readlines()
    idxs = index.readlines()
    dish_ingredients = ingredients.readlines() 

    f = open(dir_save+'tuple_dish_ingredients.txt','w')
    
    for i in range(len(idxs)):
    
        dish = dish_name[i].split('/')[0]
        ingredients = dish_ingredients[int(idxs[i])-1].replace('\n','').split(',')
    
        f.write(str(dish)+'|'+str(ingredients)+'\n')
    
    f.close()

In [3]:
if not os.path.isdir('../datasets/Recipes5k/mtannotations/'):
    os.mkdir('../datasets/Recipes5k/mtannotations/')

Generate files which contain dish-ingredient relation, dish-ingredient occurrences and the tuples of dish-ingredient labels for each recipe.

In [4]:
dir_recipe5k = '../datasets/Recipes5K/annotations/'
dir_save = '../datasets/Recipes5k/mtannotations/'

Prepr_1_Dish_Ingr(dir_recipe5k, dir_save)
Prepr_2_Dish_Ingr_Occ(dir_save,dir_save)
tuple_dish_ingr(dir_recipe5k,dir_save)

## 1 - Probability of Ingredient given a Dish (Top-Down)

## Dish Counts

In [5]:
train_images = open('../datasets/Recipes5k/annotations/train_images.txt','r')
images_list = train_images.readlines()

dishes = [image.split('/')[0] for image in images_list]
dishes_df = pd.DataFrame(dishes, columns = ['Dish'])
dish_count = dishes_df.groupby(['Dish']).size().reset_index(name = 'count')
dish_count.to_csv('../datasets/Recipes5k/mtannotations/dish_count.csv')

## Specific dishes

##### Count the number of recipes for each dish

In [6]:
dish_names = pd.read_csv('../datasets/Recipes5k/mtannotations/dish_count.csv')

# desserts = ['apple_pie', 'bread_pudding', 'carrot_cake', 'cannoli','cheesecake',
#            'chocolate_cake','chocolate_ice_cream','chocolate_mousse','churros',
#            'cupcakes','donuts','frozen_yogurt','macarons','panna_cotta',
#            'pancakes','red_velvet_cake','strawberry_shortcake','tiramisu','waffles']

# desserts
sel_dishes = ['apple_pie','carrot_cake','cheesecake','chocolate_mousse','tiramisu','panna_cotta',
            'waffles','red_velvet_cake','cupcakes','frozen_yogurt']

dish_names[dish_names.Dish.isin(sel_dishes)][['Dish','count']].to_csv('../datasets/Recipes5k/mtannotations/10_dish_count.csv', index = False)

In [7]:
sel_dish_count = pd.read_csv('../datasets/Recipes5k/mtannotations/10_dish_count.csv')

##### Dish - Ingredient Occurrences

In [8]:
#load file with dish-ingredient occurrences
#obtained with Prepr_2_Dish_Ingr_Occurrences
dish_ingr_occ = pd.read_csv('../datasets/Recipes5k/mtannotations/dish_ingredient_occurrences.csv')

#select dishes
sel_dish_ingr_occ = dish_ingr_occ[dish_ingr_occ.Dish.isin(sel_dishes)]

We are using base ingredients, then the number of dish-base ingredient occurrences might be larger than the number of recipes for each dish. This is because in each recipe the same base ingredient could be used in more than one derived ingredient.

In [9]:
dishes_tbl_df = sel_dish_ingr_occ.merge(sel_dish_count, how = 'left', left_on='Dish' , right_on='Dish')

In [10]:
def Ingredient_Probability(df):
    
    if df['Occurrence']/df['count'] < 1:
        
        prob = df['Occurrence']/df['count']
        
    else:
        
        prob = 1
        
    return prob

Calculate the probability in a frequentist way

In [11]:
dishes_tbl_df['Ingredient Probability'] = dishes_tbl_df.apply(Ingredient_Probability, axis=1)

In [12]:
dishes_tbl_df = dishes_tbl_df[['Dish','Ingredient','Ingredient Probability']]

In [13]:
f = open('../datasets/Recipes5k/mtannotations/dish_ingr_probabilities.txt','w')

f.write('Dish, Ingredient, Ingredient Probability\n')

for elem in dishes_tbl_df.values:
    
    dish_ingr = (elem[0],elem[1])
    f.write(str((dish_ingr, elem[2]))+'\n')
    
f.close()

## 2 - Probability of a Dish given an Ingredient (Bottom-Up)

In [14]:
dish_ingr_df = pd.read_csv('../datasets/Recipes5k/mtannotations/dish_ingredient_relation.txt',sep=',',names=['Dish','Ingredient'])

In [15]:
dish_ingr_copy_df = dish_ingr_df.copy()
dish_ingr_copy_df = dish_ingr_copy_df[dish_ingr_copy_df.Dish.isin(sel_dishes)]
dish_ingr_copy_df['Occurrence'] = 1
#
Ingr_occurrences_df = dish_ingr_copy_df[['Ingredient','Occurrence']].groupby(['Ingredient']).count()
#
Dish_Ingr_occurrences_df = dish_ingr_copy_df[['Ingredient','Dish','Occurrence']].groupby(['Ingredient','Dish']).count()

In [16]:
Ingr_occurrences_df = Ingr_occurrences_df.rename(columns={"Occurrence":"Times Ingredient in the Ontology"})
Dish_Ingr_occurrences_df = Dish_Ingr_occurrences_df.rename(columns={"Occurrence":"Times Ingredient in the Recipe Dish"})

In [17]:
dish_ingr_df = dish_ingr_df[dish_ingr_df.Dish.isin(sel_dishes)]
dish_ingr_df = dish_ingr_df.drop_duplicates()
Ingr_Dish_df = dish_ingr_df[['Ingredient','Dish']]
Ingr_occurrence_ontology = Ingr_Dish_df.merge(Ingr_occurrences_df, how='left', left_on = 'Ingredient', right_index=True)
Ingr_Dish_occurrence_df = Ingr_occurrence_ontology.merge(Dish_Ingr_occurrences_df, how='left', left_on=['Ingredient','Dish'], right_index=True)

In [18]:
Ingr_Dish_occurrence_df[Ingr_Dish_occurrence_df.Ingredient == 'maple']

,Ingredient,Dish,Times Ingredient in the Ontology,Times Ingredient in the Recipe Dish
119,maple,apple_pie,1,1


In [19]:
def Dish_probability(df):
    
    prob = df['Times Ingredient in the Recipe Dish']/df['Times Ingredient in the Ontology']
    
    return prob

In [20]:
Ingr_Dish_Bottom_Up_Probability_df = Ingr_Dish_occurrence_df.copy()
Ingr_Dish_Bottom_Up_Probability_df['Probability Ingr-Dish bottom-up'] = Ingr_Dish_Bottom_Up_Probability_df.apply(Dish_probability,axis=1)
Ingr_Dish_Bottom_Up_Probability_df = Ingr_Dish_Bottom_Up_Probability_df[['Ingredient','Dish','Probability Ingr-Dish bottom-up']]

In [21]:
f = open('../datasets/Recipes5k/mtannotations/ingr_dish_probabilities.txt','w')

f.write('Ingredient, Dish, Ingr-Dish Prob\n')

for elem in Ingr_Dish_Bottom_Up_Probability_df.values:
    
    f.write(str(elem[0])+','+str(elem[1])+','+str(elem[2])+'\n')
    
f.close()

## 3 - Probability of Ingredients Coexistence

https://stackoverflow.com/questions/942543/operation-on-every-pair-of-element-in-a-list

In [22]:
#Functions

def all_possible_pair_excluding_duplicates_unique(list_of_values):
    
    result = [(list_of_values[p1], list_of_values[p2]) for p1 in range(len(list_of_values)) for p2 in range(p1+1,len(list_of_values)) if list_of_values[p1] != list_of_values[p2]]
    
    return result

def explode_ingredients(df, col_expl):
    #explode the ingredients of a recipe
    
    df[col_expl] = df.loc[:,col_expl].apply(lambda x: x.replace('[','').replace(']',''))
    df[col_expl] = df.loc[:,col_expl].apply(lambda x: x.replace('\'','').replace(' ','').split(','))
    
    df[col_expl] = df[col_expl].apply(all_possible_pair_excluding_duplicates_unique)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns})

    index = 0


    for i in range(len(df)):
        for j in range(len(df[col_expl].values[i])):
             
            df_out.loc[index,col_expl] = tuple(sorted(df[col_expl].values[i][j]))
            index +=1
            
    return df_out

In [23]:
#load file with dish-ingredient occurrences
dish_recipe_file = pd.read_csv('../datasets/Recipes5k/mtannotations/tuple_dish_ingredients.txt', sep='|', header = None)
dish_recipe_file.columns = ['Dish','Ingredients']
# desserts
sel_dishes = ['apple_pie','carrot_cake','cheesecake','chocolate_mousse','tiramisu','panna_cotta',
            'waffles','red_velvet_cake','cupcakes','frozen_yogurt']

dish_recipe_file = dish_recipe_file[dish_recipe_file.Dish.isin(sel_dishes)]

In [24]:
dish_recipe_file.head()

,Dish,Ingredients
0,apple_pie,"['apple', 'margarine', 'flour', 'water', 'oats..."
1,apple_pie,"['nut', 'cinnamon', 'apple', 'sugar', 'flour',..."
2,apple_pie,"['apple', 'flour', 'sugar', 'salt', 'gin', 'ci..."
3,apple_pie,"['apple', 'sugar', 'cinnamon', 'lemon', 'flour..."
4,apple_pie,"['pie', 'apple', 'cinnamon', 'water', 'vanilla..."


In [25]:
df_dish = explode_ingredients(dish_recipe_file,'Ingredients')

df_dish.head()

,Dish,Ingredients
0,apple_pie,"(apple, margarine)"
1,apple_pie,"(apple, flour)"
2,apple_pie,"(apple, water)"
3,apple_pie,"(apple, oats)"
4,apple_pie,"(apple, sugar)"


In [26]:
len(df_dish)

17109

In [27]:
df_dish.to_csv('../datasets/Recipes5k/mtannotations/pair_ingredients.csv')

## Pair Ingredients

In [28]:
df_pair_ingredients = pd.read_csv('../datasets/Recipes5k/mtannotations/pair_ingredients.csv')
df_pair_ingredients = df_pair_ingredients[['Dish','Ingredients']]
df_pair_ingredients.head()

,Dish,Ingredients
0,apple_pie,"('apple', 'margarine')"
1,apple_pie,"('apple', 'flour')"
2,apple_pie,"('apple', 'water')"
3,apple_pie,"('apple', 'oats')"
4,apple_pie,"('apple', 'sugar')"


#### 1 - Different pairs in a recipe

In [29]:
df_unique_pair_ingredients = df_pair_ingredients.drop_duplicates()
df_unique_pair_ingredients.loc[:,'count'] = 1
df_unique_pair_ingredients[['Dish','count']].groupby(['Dish']).count().rename(columns={"count":"different pairs"})
df_different_pair_ingredients = df_pair_ingredients[['Dish','Ingredients']].merge(df_unique_pair_ingredients[['Dish','count']].groupby(['Dish']).count().rename(columns={"count":"different pairs"}), how='left', left_on='Dish', right_index=True)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [30]:
df_unique_pair_ingredients[['Dish','count']].groupby(['Dish']).count().rename(columns={"count":"different pairs"})

,different pairs
Dish,
apple_pie,285
carrot_cake,510
cheesecake,731
chocolate_mousse,176
cupcakes,666
frozen_yogurt,273
panna_cotta,439
red_velvet_cake,368
tiramisu,309


#### 2 - Times a pair appears in the recipe

In [31]:
df_final_ingredients = df_pair_ingredients
df_final_ingredients.loc[:,'count'] = 1
df_final_ingredients = df_different_pair_ingredients.merge(df_final_ingredients.groupby(['Dish','Ingredients']).count().rename(columns={"count":"times of the pair"}), how='left', left_on=['Dish','Ingredients'], right_index=True)
df_final_ingredients = df_final_ingredients.drop_duplicates()
df_final_ingredients.head(10)

,Dish,Ingredients,different pairs,times of the pair
0,apple_pie,"('apple', 'margarine')",285,3
1,apple_pie,"('apple', 'flour')",285,33
2,apple_pie,"('apple', 'water')",285,6
3,apple_pie,"('apple', 'oats')",285,1
4,apple_pie,"('apple', 'sugar')",285,30
5,apple_pie,"('apple', 'cornstarch')",285,6
6,apple_pie,"('apple', 'cinnamon')",285,38
7,apple_pie,"('flour', 'margarine')",285,1
8,apple_pie,"('margarine', 'water')",285,1
9,apple_pie,"('margarine', 'oats')",285,1


### Compute the probability

Compute the probability in a frequentist way

In [32]:
def coexistence_probability(df):
    
    df['coexistence_probability'] = df['times of the pair']/df['different pairs']
    
    return df

In [33]:
df_ingredients_coexistence_probability = df_final_ingredients.apply(coexistence_probability, axis=1)
df_ingredients_coexistence_probability = df_ingredients_coexistence_probability[['Dish','Ingredients','coexistence_probability']]
df_ingredients_coexistence_probability.head()

,Dish,Ingredients,coexistence_probability
0,apple_pie,"('apple', 'margarine')",0.010526
1,apple_pie,"('apple', 'flour')",0.115789
2,apple_pie,"('apple', 'water')",0.021053
3,apple_pie,"('apple', 'oats')",0.003509
4,apple_pie,"('apple', 'sugar')",0.105263


In [34]:
f = open('../datasets/Recipes5k/mtannotations/dish_ingr_pair_coexistence_probability.txt','w')

f.write('Dish, Ingredients Pair, Coexistence Probability\n')

for elem in df_ingredients_coexistence_probability.values:
    
    f.write(str(((elem[0],elem[1]), elem[2]))+'\n')
    
f.close()

#### Create dishes list and concepts list needed to build the ontology

Save a file with the list of dishes and another with the total concepts

In [35]:
#Create dish list file
f = open('../datasets/Recipes5k/mtannotations/dish_list.txt','w')

for dish in sel_dishes:
  
  f.write(dish)
  f.write('\n')
  
f.close()

In [36]:
dir_recipe5k = '../datasets/Recipes5K/'

dir_images = dir_recipe5k + 'images/'
dir_annotations = dir_recipe5k + 'annotations/'

train_dishes = open(dir_annotations+'train_images.txt','r')
train_index = open(dir_annotations+'train_labels.txt','r')
ingredients = open(dir_annotations+'ingredients_simplified_Recipes5k.txt','r')

val_dishes = open(dir_annotations+'val_images.txt','r')
val_index = open(dir_annotations+'val_labels.txt','r')

test_dishes = open(dir_annotations+'test_images.txt','r')
test_index = open(dir_annotations+'test_labels.txt','r')

train_dish_name = train_dishes.readlines()
train_idxs = train_index.readlines()

val_dish_name = val_dishes.readlines()
val_idxs = val_index.readlines()

test_dish_name = test_dishes.readlines()
test_idxs = test_index.readlines()

dish_ingredients = ingredients.readlines() 


ingr_file = open('../datasets/Recipes5k/mtannotations/prep_list_ingredients_img.txt','w')

ingr_labels = []

for i in range(len(train_idxs)):
  
  dish = train_dish_name[i].split('/')[0]
        
  if dish in sel_dishes:  

    labels = dish_ingredients[int(train_idxs[i])-1][:-1].split(',')
  
    if labels not in ingr_labels:
  
      ingr_labels.append(labels)
    
for i in range(len(val_idxs)):
  
  dish = val_dish_name[i].split('/')[0]
        
  if dish in sel_dishes:  

    labels = dish_ingredients[int(val_idxs[i])-1][:-1].split(',')
  
    if labels not in ingr_labels:
  
      ingr_labels.append(labels)
    
for i in range(len(test_idxs)):

  dish = test_dish_name[i].split('/')[0]
        
  if dish in sel_dishes:  
  
    labels = dish_ingredients[int(test_idxs[i])-1][:-1].split(',')
  
    if labels not in ingr_labels:
    
      ingr_labels.append(labels)
    

for labels in ingr_labels:

  ingr_file.write(str(labels))
  ingr_file.write('\n')
  
ingr_file.close()

n_unique_ingr_list = len(ingr_labels)
ingr_labels = [str(labels) for labels in ingr_labels]  
dict_ingr_labels = dict(zip(ingr_labels, range(n_unique_ingr_list)))

f = open("../datasets/Recipes5k/mtannotations/list_ingredients_img.txt", "w")

with open("../datasets/Recipes5k/mtannotations/prep_list_ingredients_img.txt", "r") as prev:
  for line in prev:
    f.write(line.replace('[','').replace(']','').replace('\'',''))
    #f.write('\n')
f.close()

ingredient_list = []
ingredients = []
with open("../datasets/Recipes5k/mtannotations/list_ingredients_img.txt", "r") as f:
    for line in f:
        #print(line.strip())
        ingredients = ingredients + line.strip().split(", ")
        ingredient_list.append(line.strip())

ingredients = list(set(ingredients))

concepts_list = sel_dishes + ingredients

with open("../datasets/Recipes5k/mtannotations/concepts_list.txt","w") as f:
  for concept in concepts_list:
    f.write(concept)
    f.write('\n')